In [28]:
%%time
import cv2
import os
import numpy as np
from PIL import Image
import glob
from tqdm import tqdm_notebook as tqdm

step_sec = 1
os.makedirs('frames', exist_ok=True)

for fname in tqdm(glob.glob('downloaded/*.mp4')[:1]):
    fname_w_o_extension = fname[:-4]
    video = cv2.VideoCapture(fname)
    fps = video.get(cv2.CAP_PROP_FPS)
    time_length = video.get(cv2.CAP_PROP_FRAME_COUNT)
    frame_seq = 0
    count_sec = 0
    while True:         
        frame_no = (frame_seq /(time_length*fps))
        video.set(cv2.CAP_PROP_POS_FRAMES, fps*count_sec)
        ret, frame = video.read()
        if ret:
            cv2.imwrite(f"frames/{fname_w_o_extension}_{count_sec}.jpg", frame)
            print('saved')
            frame_seq += fps
            count_sec += step_sec
        else:
            break

<timed exec>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1 [00:00<?, ?it/s]

saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
save

In [36]:
!rm -rf frames/*

In [ ]:
for fname in tqdm(os.listdir('downloaded')):
    if fname.endswith('.mp4'):
        fname_w_o_extension = fname[:-4]
        video = cv2.VideoCapture(f"downloaded/{fname}")
        fps = video.get(cv2.CAP_PROP_FPS)
        time_length = video.get(cv2.CAP_PROP_FRAME_COUNT)
        frame_seq = 0
        count_sec = 0
        while True:         
            frame_no = (frame_seq /(time_length*fps))
            video.set(cv2.CAP_PROP_POS_FRAMES, fps*count_sec)
            ret, frame = video.read()
            if ret:
                cv2.imwrite(f"frames/{fname_w_o_extension}_{count_sec}.jpg", frame)
                frame_seq += fps
                count_sec += 1
            else:
                break

/tmp/ipykernel_11682/17089869.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for fname in tqdm(os.listdir('downloaded')):


  0%|          | 0/151 [00:00<?, ?it/s]

[h264 @ 0x61c2c40] Invalid NAL unit size (8046 > 4909).
[h264 @ 0x61c2c40] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3600c41: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36013d7: partial file
[h264 @ 0x61c2c40] Invalid NAL unit size (8046 > 4909).
[h264 @ 0x61c2c40] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3600c41: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36013d7: partial file
[h264 @ 0x61c2c40] Invalid NAL unit size (8046 > 4909).
[h264 @ 0x61c2c40] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3600c41: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36013d7: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36381d4: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36381d4: partial file
[mov,mp4,m4a,3gp,

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3600c41: partial file
[h264 @ 0x61c2c40] Invalid NAL unit size (8046 > 4909).
[h264 @ 0x61c2c40] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36013d7: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3767c6b: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3767c6b: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36a96e1: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36a96e1: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36a96e1: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36a96e1: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36a96e1: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36381d4: partial file
[h264 @ 0x61c2c40] Invalid NAL unit size (8046 > 4909).
[h264 @ 0x61c2c40] Error splitting th

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3600c41: partial file
[h264 @ 0x61c2c40] Invalid NAL unit size (8046 > 4909).
[h264 @ 0x61c2c40] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36013d7: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3852aed: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3852aed: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3852aed: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x37c0d82: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x37c0d82: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x37c0d82: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3767c6b: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3767c6b: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36a96e1: partial file
[h264 @ 0x61c2

...

[h264 @ 0x5801f40] Invalid NAL unit size (8046 > 4909).
[h264 @ 0x5801f40] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3600c41: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36013d7: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x45fd6ef: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x4351056: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x42bc8bc: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x41ce42e: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x4059e0b: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3e5f558: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3bb5257: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x37c0d82: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3600c41: partial file
[h264 @ 0x5801

[h264 @ 0x5801f40] Invalid NAL unit size (8046 > 4909).
[h264 @ 0x5801f40] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3600c41: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36013d7: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x4741095: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x4741095: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x4741095: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x4741095: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x45fd6ef: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x42bc8bc: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x4147951: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3faec84: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 1, offset 0x3c5f4dd: partial file
[mov,mp4,m4a,3

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3600c41: partial file
[h264 @ 0x61c2c40] Invalid NAL unit size (8046 > 4909).
[h264 @ 0x61c2c40] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36013d7: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x4741095: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x4741095: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x45fd6ef: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x4059e0b: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 1, offset 0x3cfd9e5: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x39654eb: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3600c41: partial file
[h264 @ 0x5796cc0] Invalid NAL unit size (8046 > 4909).
[h264 @ 0x5796cc0] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 

[h264 @ 0x61c2c40] Invalid NAL unit size (8046 > 4909).
[h264 @ 0x61c2c40] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3600c41: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36013d7: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x4741095: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x4351056: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x41ce42e: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3e5f558: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3a4e102: partial file
[h264 @ 0x5840e00] Invalid NAL unit size (8046 > 4909).
[h264 @ 0x5840e00] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3600c41: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36013d7: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 

[h264 @ 0x61c2c40] Invalid NAL unit size (8046 > 4909).
[h264 @ 0x61c2c40] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3600c41: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x36013d7: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x495f24f: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x495f24f: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 1, offset 0x49102a8: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x45fd6ef: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x42bc8bc: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x4059e0b: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 0x3bb5257: partial file
[h264 @ 0x61c2c40] Invalid NAL unit size (8046 > 4909).
[h264 @ 0x61c2c40] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6435d00] stream 0, offset 

In [32]:
! pwd

/home/vkhack/videogen


In [16]:
fname, time_length

('downloaded/dNCWe_6HAM8.mp4', 4088.0)

In [14]:
ret

False

In [12]:
! ls

app.py				     img
clip_embedder.py		     indexer
cut_frames_and_calculate_embs.ipynb  indexer.ipynb
data				     model.py
Dockerfile			     nmslib_indexer.py
download_data.ipynb		     __pycache__
downloaded			     README.md
frames_prev			     requirements.txt
frame_vectors_prev		     videos
generate_video_cut_queries.py	     wget-log


In [10]:
! ls frames/*

ls: cannot access 'frames/*': No such file or directory


In [ ]:
%%time
# save embeddings
from clip_embedder import ClipEmbedder

embedder = ClipEmbedder()
device = 'cpu'

for fname in glob.glob('frames/*.jpg'):
    fname_w_o_extension = fname[:-4]
    image = embedder.preprocess(Image.open(f"frames/{fname}")).unsqueeze(0).to(device)
    image_features = embedder.encode_image(image)
    np.save(f"frame_vectors/{fname_w_o_extension}.npy", image_features.detach().numpy(), allow_pickle=False)

In [30]:
from scipy.spatial import distance

ModuleNotFoundError: No module named 'scipy'